# 第4章 レポーティングする仕組みを構築する10本ノック

In [14]:
using DataFrames
using DataFramesMeta
using CSV
using Glob
using XLSX

## 31

In [13]:
m_store_df = CSV.read("../data/4章/m_store.csv", DataFrame)
m_area_df = CSV.read("../data/4章/m_area.csv", DataFrame)
ifns = glob("../data/4章/tbl_order_??????.csv")
order_all_df = DataFrame()
for ifn_i in ifns
    tmp_df = CSV.read(ifn_i, DataFrame)
    append!(order_all_df, tmp_df)
end
order_all_df = @chain order_all_df begin
    @rsubset(:store_id != 999)
    leftjoin(m_store_df, on = :store_id)
    leftjoin(m_area_df, on = :area_cd)
    @rtransform begin
        :takeout_name = begin
            if :takeout_flag == 0
                return "デリバリー"
            elseif :takeout_flag == 1
                return "お持ち帰り"
            else
                return missing
            end
        end
        :status_name = begin
            if :status == 0
                return "受付"
            elseif :status == 1
                return "お支払済"
            elseif :status == 2
                return "お渡し済"
            elseif :status == 9
                return "キャンセル"
            else
                return missing
            end
        end
    end
end

Row,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
,Int64,Int64,String15,Int64,Int64,String31,String31,Int64,Int64,Int64,String31?,String3?,String15?,String15?,String,String
1,79339111,49,C26387220,50,67393872,2020-04-01 11:00:00,2020-04-01 11:18:00,1,4144,1,浅草店,TK,東京,東京,お持ち帰り,お支払済
2,18941733,85,C48773811,26,91834983,2020-04-01 11:00:00,2020-04-01 11:22:00,0,2877,2,目黒店,TK,東京,東京,デリバリー,お渡し済
3,56217880,76,C66287421,36,64409634,2020-04-01 11:00:00,2020-04-01 11:15:00,0,2603,2,本郷店,TK,東京,東京,デリバリー,お渡し済
4,28447783,190,C41156423,19,73032165,2020-04-01 11:00:00,2020-04-01 11:16:00,0,2732,2,栃木店,TO,北関東,栃木,デリバリー,お渡し済
5,32576156,191,C54568117,71,23281182,2020-04-01 11:00:00,2020-04-01 11:53:00,0,2987,2,伊勢崎店,GU,北関東,群馬,デリバリー,お渡し済
6,78384157,16,C91828823,88,38723994,2020-04-01 11:00:00,2020-04-01 11:19:00,1,2122,1,蒲田店,TK,東京,東京,お持ち帰り,お支払済
7,75031687,144,C58120290,63,82478302,2020-04-01 11:00:00,2020-04-01 11:22:00,0,2308,2,流山店,CH,千葉,千葉,デリバリー,お渡し済
8,83880121,77,C08085038,0,63603896,2020-04-01 11:00:00,2020-04-01 11:25:00,0,2112,2,大塚店,TK,東京,東京,デリバリー,お渡し済
9,26741317,22,C24382968,72,44038720,2020-04-01 11:00:00,2020-04-01 11:15:00,1,3120,9,国立店,TK,東京,東京,お持ち帰り,キャンセル


In [16]:
XLSX.openxlsx("test.xlsx", mode="w") do xf
    sheet = xf[1]
    sheet["A1"] = "これは書き込みテストです．"
end

"これは書き込みテストです．"

In [17]:
XLSX.openxlsx("test.xlsx", mode="r") do xf
    sheet = xf[1]
    println(sheet["A1"])
end

これは書き込みテストです．


In [39]:
store_id = 1
@chain order_all_df begin
    @rsubset begin
        :store_id == store_id
        :status in [1, 2]
    end
    groupby([:status, :status_name, :store_name])
    @combine(:total_amount = sum(:total_amount))
    sort(:status)
end

Row,status,status_name,store_name,total_amount
,Int64,String,String31?,Int64
1,1,お支払済,昭島店,2255499
2,2,お渡し済,昭島店,6749036


In [40]:
store_id = 1
output_df = @chain order_all_df begin
    @rsubset begin
        :store_id == store_id
        :status in [1, 2]
    end
    @select(:order_accept_date, :customer_id, :total_amount, :takeout_name, :status_name)
end

Row,order_accept_date,customer_id,total_amount,takeout_name,status_name
,String31,String15,Int64,String,String
1,2020-04-01 11:09:09,C25851661,2471,デリバリー,お渡し済
2,2020-04-01 11:49:49,C80269937,2615,お持ち帰り,お支払済
3,2020-04-01 12:41:41,C77508455,3931,デリバリー,お渡し済
4,2020-04-01 13:45:45,C71855263,3586,デリバリー,お渡し済
5,2020-04-01 13:50:50,C85259317,2808,お持ち帰り,お支払済
6,2020-04-01 14:22:22,C76724964,1882,お持ち帰り,お支払済
7,2020-04-01 15:06:06,C50758423,5100,デリバリー,お渡し済
8,2020-04-01 15:20:20,C97884595,2064,お持ち帰り,お支払済
9,2020-04-01 16:07:07,C62881190,2877,デリバリー,お渡し済


In [41]:
store_title = "1_昭島店"
XLSX.openxlsx("$(store_title).xlsx", mode="w") do xf
    sheet = xf[1]
    XLSX.rename!(sheet, store_title) 
    sheet["A1"] = "$(store_title) 売上データ"
    XLSX.writetable!(sheet, output_df; anchor_cell=XLSX.CellRef("B3"))
end

## 32